In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Load and preprocess the data
df = pd.read_csv('combined_output.csv')
df2 = df.drop(' Label', axis=1)
df2 = df2.dropna()

# Replace infinite values with NaN and drop rows with NaNs
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df2_cleaned = df2.dropna()

# Select only numeric columns, keeping 'Label Num' for later comparison
numeric_df = df2_cleaned.select_dtypes(include=[np.number])

# Extract the 'Label Num' for the ground truth comparison later
labels = numeric_df['Label Num']

# Drop 'Label Num' column from the features for clustering
numeric_df = numeric_df.drop('Label Num', axis=1)

# Convert to NumPy array and scale the data
X = numeric_df.to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train_labels, y_test_labels = train_test_split(X_scaled, labels, test_size=0.2, random_state=42, stratify=labels)

# Apply SMOTE to balance the classes in the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train_labels)

# Define the pipeline with KMeans
pipeline = Pipeline([
    ('kmeans', KMeans())
])

# Setup the grid search parameters for KMeans
param_grid = {
    'kmeans__n_clusters': [5, 7, 9, 11],
    'kmeans__n_init': [10, 20],
    'kmeans__algorithm': ['lloyd', 'elkan']
}

# Davies-Bouldin score as the scoring metric
def davies_bouldin_scorer(estimator, X):
    labels = estimator.predict(X)
    return -davies_bouldin_score(X, labels)  # Minimize Davies-Bouldin score

# Initialize the grid search for KMeans clustering
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=davies_bouldin_scorer,
    cv=5,
)

# Perform grid search on the training dataset
grid_search.fit(X_train_balanced)

# Make predictions on the test set
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the Davies-Bouldin score for the test set
db_score_test = davies_bouldin_score(X_test, y_pred)

# Print the Davies-Bouldin score for the test set
print("Davies-Bouldin Score for the test data:", db_score_test)

# Create a DataFrame to examine the distribution between clusters and actual labels
data_with_predictions = pd.DataFrame({
    'Prediction': y_pred,  # Predicted clusters
    'Actual': y_test_labels.reset_index(drop=True)  # Original labels (0 or 1)
})

# Group by cluster prediction and actual label, then count occurrences
distribution = pd.crosstab(data_with_predictions['Prediction'], data_with_predictions['Actual'])

# Print the distribution matrix
print(distribution)

Davies-Bouldin Score for the test data: 1.5269311669346415
Actual           0      1
Prediction               
0           303290  37093
1            14342  33100
2           131286  25208
3               10      0
4             5337  15910
